In [ ]:
### import torch
from torchvision import models

pretrained_model = models.resnet50(weights = 'DEFAULT', progress = True)

In [2]:
n = 100 #let n = the number of types of plants

In [3]:
# changes fully connected/classifier layer to new layer for us to train
pretrained_model.fc = torch.nn.Linear(2048, 2)

In [7]:
# Currently no in use
import torch.nn as nn
# build custom softmax module
class Softmax(nn.Module):
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        self.linear = nn.Linear(n_inputs, n_outputs)
 
    def forward(self, x):
        pred = self.linear(x)
        return pred

In [8]:
# Currently no in use
#add softmax to model
class MyModel(nn.Module):
    def __init__(self, pretrained_model):
        super(MyModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.last_layer = Softmax(1000, n) # add how many nodes as input and output

    def forward(self, x):
        return self.last_layer(self.pretrained_model(x))

model = MyModel(pretrained_model)

In [11]:
#freeze model except fc layer
for param in pretrained_model.parameters():
    param.requires_grad = False

for param in pretrained_model.fc.parameters():
    param.requires_grad = True

In [12]:
criterion = torch.nn.CrossEntropyLoss() #could write this out ourselves
# need to find an optimizer or make onefor custom softmax function
optimizer = torch.optim.SGD(pretrained_model.fc.parameters(), lr=0.001, momentum=0.9)

In [12]:
#import data
from PIL import Image
import os

def getLabel(s):
    labels = {1 : 'Ulmus carpinifolia', 
                2 : 'Acer', 
                3 : 'Salix aurita', 
                4 : 'Quercus', 
                5 : 'Alnus incan', 
                6 : 'Betula pubescens', 
                7 : 'Salix alba \'Sericea\'', 
                8 : 'Populus tremula', 
                9 : 'Ulmus glabra', 
                10 : 'Sorbus aucuparia', 
                11 : 'Salix sinerea', 
                12 : 'Populus', 
                13 : 'Tilia', 
                14 : 'Sorbus intermedia', 
                15 : 'Fagus silvatica'}
    return labels.get(int(s))

im = [] #images
la = [] #labels
data_dir = "/Users/stevenli/SigAida/data/images/" #change to data directory
for f in os.listdir(data_dir):
    im.append(Image.open(data_dir + f))
    la.append(getLabel(f[:-9].replace('l',''))) #removes last 9 letters replaces l with blank then gets label

print(la)
image1 = im[1]
image1.show()




['Populus tremula', 'Ulmus glabra', 'Ulmus glabra', 'Populus tremula', 'Populus tremula', 'Ulmus glabra', 'Sorbus aucuparia', 'Salix sinerea', 'Salix sinerea', 'Sorbus aucuparia', 'Salix aurita', 'Acer', 'Acer', 'Salix aurita', 'Ulmus carpinifolia', 'Ulmus carpinifolia', 'Populus', 'Ulmus carpinifolia', 'Tilia', 'Tilia', 'Populus', 'Alnus incan', 'Quercus', 'Betula pubescens', 'Fagus silvatica', 'Sorbus intermedia', "Salix alba 'Sericea'", 'Sorbus intermedia', 'Fagus silvatica', 'Betula pubescens', "Salix alba 'Sericea'", "Salix alba 'Sericea'", 'Betula pubescens', "Salix alba 'Sericea'", 'Betula pubescens', 'Betula pubescens', "Salix alba 'Sericea'", 'Sorbus intermedia', "Salix alba 'Sericea'", 'Fagus silvatica', 'Fagus silvatica', 'Betula pubescens', 'Sorbus intermedia', 'Quercus', 'Alnus incan', 'Tilia', 'Populus', 'Ulmus carpinifolia', 'Populus', 'Tilia', 'Ulmus carpinifolia', 'Ulmus carpinifolia', 'Acer', 'Salix aurita', 'Salix aurita', 'Acer', 'Salix sinerea', 'Sorbus aucuparia',

In [ ]:
#some preprocessing, randomize data, split data, load into dataloaders
def create_splits(spleens, split_ratio):
    test = []
    train = []
    val = []
    train_val, test = split_array(spleens, train_size = (split_ratio[0] + split_ratio[1]), test_size = split_ratio[2])
    train_size = round(split_ratio[0] / (split_ratio[0]+ split_ratio[1]), 2)
    val_size = round(split_ratio[1] / (split_ratio[0]+ split_ratio[1]), 2)
    train, val = split_array(train_val, train_size=train_size, test_size = val_size)
    return train, val, test

train, val, test = create_splits(spleens, split_ratio=(0.8, 0.1, 0.1))

In [13]:
# should probably make some changes here to get what we want

import torchvision.datasets as datasets
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224), # change to waht data should be
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print some statistics
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


NameError: name 'device' is not defined